#### Loading libraries

In [1]:
# Load tensorflow
import tensorflow as tf
# Below command is to avoid the known bug which prevents computation on some GPU devices
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
tf.config.experimental.set_memory_growth(physical_devices[0], True)
# Load preprocessing tools
from scipy.ndimage.filters import gaussian_filter
from tensorflow.keras.utils import to_categorical
from PIL import Image
# Load model building blocks
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
# Load pre-trained model library
from tensorflow.keras import applications
# Load miscelaneous libraries
import numpy as np
import os
import matplotlib.pylab as plt
import gc
# Settings to set the max output of numpy to infty
# import sys
# np.set_printoptions(threshold=sys.maxsize)

#### Define image processor and data generator

In [9]:
# Define gaussian blur class
class GaussBlur:
    def __init__(self, radius):
        self.radius = radius
    def blur(self, image):
        return gaussian_filter(image, sigma = self.radius)

In [10]:
# Define class for data generation
class DataGenerator:
    'Generates data for Keras'
    def __init__(self, list_image_paths = None,  
                 dim = None,
                 n_channels = 3, 
                 rescale = 1, 
                 preprocessing_func = None):
        'Initialization'
        self.list_image_paths = list_image_paths
        self.dim = dim
        self.n_channels = n_channels
        self.rescale = rescale
        self.preprocessing_func = preprocessing_func

    def generate(self):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        
        # Initialisation
        X = np.empty((len(self.list_image_paths), *self.dim, self.n_channels))
        
        # Generate data
        for i, image_path in enumerate(self.list_image_paths):
            # Load image and transform
            image = Image.open(os.path.join(image_path))
            if self.dim is not None:
                image = image.resize(self.dim, resample = Image.NEAREST)
            image = np.array(image)[:, :, :self.n_channels]
            image = image * self.rescale
            if self.preprocessing_func is not None:
                image = self.preprocessing_func(image)
            # Store sample
            X[i,] = image

        return X

In [11]:
# Define sorter of image names in order by image number (default is alphanumric)
def sorter(item):
    # Since highest marks first, least error = most marks
    radius = float(item[1 : item.find('_')])
    num_img = int(item[item.find('g') + 1 : item.find('j') - 1])
    return (radius, num_img)

#### Sort the labels

In [12]:
# Define algorithm for reordering the data based on the index array
def reorder(ind, arr):
    new_arr = np.zeros(arr.shape)
    for i in range(len(arr)):
        new_arr[i] = arr[ind[i]]
    return new_arr  

def order_labels(angles_inner, mags, angles_tang):
    # Initialise variables
    y_m = dict()
    y_ai = dict()
    y_at = dict()
    # Load the labels (contact angles, magnitudes, tangential angles)
    for i in range(5):
        y_m[i] = np.load(mags[i])
        y_ai[i] = np.load(angles_inner[i])
        y_at[i] = np.load(angles_tang[i])

    y_m_reord = dict()
    y_ai_reord = dict()
    y_at_reord = dict()
    # Perform reording
    for i in range(5):
        y_ai_reord[i] = np.sort(y_ai[i], axis = -1) 
        ind_matrix = np.argsort(y_ai[i], axis = -1)
        y_m_reord[i] = np.array([reorder(ind_matrix[j,:], y_m[i][j,:]) for j in range(y_m[i].shape[0])])
        y_at_reord[i] = np.array([reorder(ind_matrix[j,:], y_at[i][j,:]) for j in range(y_at[i].shape[0])])

    # Save data
    for i in range(5):
        np.save(angles_inner[i], y_ai_reord[i])
        np.save(angles_tang[i], y_at_reord[i])
        np.save(mags[i], y_m_reord[i])

In [13]:
# Reorder labels test
angles_inner = []
mags = []
angles_tang = []
for k in range(5):
    i = k + 2
    path = os.path.join(os.getcwd(), 'labels', 'test', str(i))
    angles_inner.append(os.path.join(path, 'angles_inner.npy'))
    mags.append(os.path.join(path, 'mags.npy'))
    angles_tang.append(os.path.join(path, 'angles_tang.npy'))

order_labels(angles_inner, mags, angles_tang)

#### Load the data

In [14]:
# Extract image paths
img_paths = {}
for k in range(5):
    i = k + 2
    path_prefix = os.path.join(os.getcwd(), 'image_data', 'test', str(i))
    img_paths[i] = [os.path.join(path_prefix, name) for name in sorted(os.listdir(path_prefix), key = sorter)]

# Apply data generators
gaussblur = GaussBlur(1)
params = {'dim': (128, 128), 
          'n_channels': 3, 
          'rescale': 1 / 255, 
          'preprocessing_func': gaussblur.blur
          }
test_generator = {}
for k in range(5):
    i = k + 2
    test_generator[i] = DataGenerator(img_paths[i], **params)
    
# Load images (for all number of forces) as matrix
X = dict()
for k in range(5):
    i = k + 2
    X[i] = test_generator[i].generate()

In [15]:
# Load angle labels
path_prefix = os.path.join(os.getcwd(), 'labels', 'test')
y_lai = dict()
for k in range(5):
    i = k + 2
    path = os.path.join(path_prefix, str(i))
    y_lai[i] = np.load(os.path.join(path,'angles_inner.npy'))

In [16]:
# Load models
models_path = os.path.join(os.getcwd(), 'models')

model_class = load_model(os.path.join(models_path, 'vgg19_num_forces.h5'))                       

In [17]:
# predict class
predicted_class = dict()
indices_correct_class = dict()
for k in range(5):
    i = k + 2
    predicted_class[i] = model_class.predict(X[i]).argmax(-1)+2
    indices_correct_class[i] = np.argwhere(predicted_class[i] == i).flatten().tolist()

In [18]:
models_ai = dict()                         
for k in range(5):
     i = k + 2
     models_ai[i] = load_model(os.path.join(models_path, 'InceptionResNetV2_angles_inner_'+str(i)+'.h5'))

In [19]:
for k in range(5):
    i = k + 2
    print("Num forces: ", i)
    loss = models_ai[i].evaluate(X[i][indices_correct_class[i], ], y_lai[i][indices_correct_class[i], ])

Num forces:  2
125/125 [==============================] - 5s 40ms/step - loss: 0.0262 - mean_absolute_error: 0.0262
Num forces:  3
125/125 [==============================] - 5s 40ms/step - loss: 0.1003 - mean_absolute_error: 0.1003
Num forces:  4
125/125 [==============================] - 5s 40ms/step - loss: 0.1748 - mean_absolute_error: 0.1748
Num forces:  5
125/125 [==============================] - 5s 39ms/step - loss: 0.2309 - mean_absolute_error: 0.2309
Num forces:  6
125/125 [==============================] - 137s 1s/step - loss: 0.1839 - mean_absolute_error: 0.1839
